In [ ]:
import json
import pandas as pd
from IPython.display import display, Markdown

def read_experiment_data(file_path):
    """读取实验数据文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def extract_experiment_info(experiments):
    """提取实验信息"""
    results = []

    for exp in experiments:
        config = exp.get("config", {})
        result = exp.get("result", {})

        # 提取所需的参数
        edge_cloud_bandwidth = config.get("edge_cloud_bandwidth", 0)
        edge_end_bandwidth = config.get("edge_end_bandwidth", 0)
        eval_mode = result.get("eval_mode", "")
        target_forward_times = result.get("target_forward_times", 0)
        wall_time = result.get("wall_time", 0)
        communication_time = result.get("communication_time", 0)
        edge_cloud_data_bytes = result.get("edge_cloud_data_bytes", 0)
        throughput = result.get("throughput", 0)
        comm_energy = result.get("comm_energy", 0)

        results.append(
            {
                "Edge-Cloud 带宽 (Mbps)": edge_cloud_bandwidth,
                "Edge-End 带宽 (Mbps)": edge_end_bandwidth,
                "方法": eval_mode,
                "目标模型 Forward 次数": target_forward_times,
                "挂钟时间 (s)": round(wall_time, 2),
                "通信时间 (s)": round(communication_time, 2),
                "Edge-Cloud 数据传输量 (bytes)": edge_cloud_data_bytes,
                "Token 输出速率 (tokens/s)": round(throughput, 2),
                "通信能量": comm_energy,
            }
        )

    return results


def generate_markdown_table(data):
    """生成 markdown 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 生成 markdown 表格
    markdown_table = df_sorted.to_markdown(index=False, tablefmt="github")
    return markdown_table


def get_pandas_table(data):
    """生成 pandas DataFrame 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )
    return df_sorted

def generate_grouped_tables(data):
    """按方法分组生成多个表格"""
    df = pd.DataFrame(data)

    # 按方法分组
    grouped = df.groupby("方法")

    all_tables = []

    for method, group_df in grouped:
        # 按Edge-Cloud带宽降序排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## {method}\n\n{table}\n")

    return "\n".join(all_tables)


def jupyter_display(experiment_data):
    """
    一个专门为 Jupyter Notebook 设计的主函数，
    用于读取、处理数据并以美观的表格形式显示结果。
    """
    # 创建
    df = pd.DataFrame(experiment_data)
    # --- 1. 显示统一排序的表格 ---
    # 使用 display(Markdown(...)) 来渲染标题
    display(Markdown("# 实验结果对比表（按方法和带宽排序）"))

    # 对 DataFrame 进行排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 直接显示 DataFrame 对象，Jupyter 会将其渲染为 HTML 表格
    display(df_sorted)
    # --- 2. 显示按方法分组的表格 ---
    display(Markdown("# 实验结果对比表（按方法分组）"))

    # 按方法进行分组
    grouped = df.groupby("方法")
    # 遍历每个分组并分别显示
    for method, group_df in grouped:
        # 显示分组标题
        display(Markdown(f"## {method}"))

        # 对当前分组的 DataFrame 进行排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 显示排序后的分组 DataFrame
        display(group_df_sorted)


def main():
    # 读取实验数据
    file_path = "experiment_summary_20250920_145859.json"
    experiments = read_experiment_data(file_path)
    experiment_data = extract_experiment_info(experiments)
    unified_table = generate_markdown_table(experiment_data)
    grouped_tables = generate_grouped_tables(experiment_data)


    jupyter_display(experiment_data)

    # 保存到文件
    with open("experiment_results_table.md", "w", encoding="utf-8") as f:
        f.write("# 实验结果对比表（按方法和带宽排序）\n\n")
        f.write(unified_table)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("# 实验结果对比表（按方法分组）\n\n")
        f.write(grouped_tables)

    print("\n表格已保存到 experiment_results_table.md 文件中")


if __name__ == "__main__":
    main()

# 实验结果对比表（按方法和带宽排序）

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
1,34.60,563.0,dist_spec,5327,639.63,8.87,38381416,32.02,0.0
5,25.00,350.0,dist_spec,5327,637.92,12.28,38381416,32.10,0.0
9,15.00,200.0,dist_spec,5327,652.72,20.47,38381416,31.38,0.0
13,5.00,100.0,dist_spec,5327,675.93,61.41,38381416,30.30,0.0
17,0.14,33.2,dist_spec,5327,2819.42,2193.22,38381416,7.26,0.0
0,34.60,563.0,dist_split_spec,5327,608.32,0.62,2665788,33.67,0.0
4,25.00,350.0,dist_split_spec,5327,613.89,0.85,2665788,33.36,0.0
8,15.00,200.0,dist_split_spec,5327,612.05,1.42,2665788,33.46,0.0
11,5.00,100.0,dist_split_spec,5327,613.57,4.27,2665788,33.38,0.0
15,0.14,33.2,dist_split_spec,5327,779.65,152.33,2665788,26.27,0.0


# 实验结果对比表（按方法分组）

## dist_spec

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
1,34.60,563.0,dist_spec,5327,639.63,8.87,38381416,32.02,0.0
5,25.00,350.0,dist_spec,5327,637.92,12.28,38381416,32.10,0.0
9,15.00,200.0,dist_spec,5327,652.72,20.47,38381416,31.38,0.0
13,5.00,100.0,dist_spec,5327,675.93,61.41,38381416,30.30,0.0
17,0.14,33.2,dist_spec,5327,2819.42,2193.22,38381416,7.26,0.0


...

## tridecoding

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
3,34.60,563.0,tridecoding,4605,894.94,1.03,4094290,23.81,0.0
7,25.00,350.0,tridecoding,4605,889.93,1.45,4094290,23.94,0.0
12,15.00,200.0,tridecoding,4605,899.82,2.43,4094290,23.68,0.0
16,5.00,100.0,tridecoding,4605,869.59,7.04,4094290,24.50,0.0
19,0.14,33.2,tridecoding,4605,1090.32,235.43,4094290,19.54,0.0


## uncertainty_decoding

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
2,34.60,563.0,uncertainty_decoding,577,6946.70,6267.51,27307432,2.95,0.0
6,25.00,350.0,uncertainty_decoding,577,6947.62,6269.94,27307432,2.95,0.0
10,15.00,200.0,uncertainty_decoding,577,6954.13,6275.76,27307432,2.95,0.0
14,5.00,100.0,uncertainty_decoding,577,6993.25,6304.89,27307432,2.93,0.0
18,0.14,33.2,uncertainty_decoding,577,8498.79,7821.62,27307432,2.41,0.0



表格已保存到 experiment_results_table.md 文件中
